In [4]:
import sys, io
import math
import numpy as np
import pandas as pd
from scipy.optimize import nnls
from fractions import Fraction
from sympy import factorint
from itertools import product
from typing import List, Optional, Tuple
import random


In [5]:
def T(n):
    if n & 1 == 0:
        return n//2
    else:
        return (3*n + 1)//2
#
def S_T(n):
    if n == 1:
        return "1"

    S = ""
    while n != 1:
        if n & 1 == 0:
            n = n//2
            S = S + "1"
        else:
            n = (3*n + 1)//2
            S = S + "0"
    return S
#

In [6]:
def Ay_S(s):
    rank = len(s) + 2    
    A = np.zeros((rank,rank))
    y = np.zeros((rank))
    for row in range(rank-3):
        if s[row-2] == "0":
            a_val = -1.0
            y_val = 0.0
        else:
            a_val = -3.0
            y_val = 1.0
        A[row][row] = a_val
        A[row][row+1] = 2.0
        y[row] = y_val
    #
    # Last 3 rows are always the same
    row = rank - 3
    A[row][row] = -1
    A[row][row+1] = 2
    y[row] = 0
    row = rank - 2
    A[row][row] = -3
    A[row][row+1] = 2
    y[row] = 1
    row = rank - 1
    A[row][row] = 1
    A[row][row-2] = -1
    y[row] = 0
    
    return A, y
#

In [7]:
Ay_S("00111")

(array([[-3.,  2.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -3.,  2.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  2.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  2.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  2.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -3.,  2.],
        [ 0.,  0.,  0.,  0., -1.,  0.,  1.]]),
 array([1., 1., 0., 0., 0., 1., 0.]))

In [8]:
def solve_Ay_S(s):
    A, y = Ay_S(s)
    return A, np.linalg.solve(A, y), y
#

In [9]:
solve_Ay_S("00111")

(array([[-3.,  2.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -3.,  2.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  2.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  2.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  2.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -3.,  2.],
        [ 0.,  0.,  0.,  0., -1.,  0.,  1.]]),
 array([3., 5., 8., 4., 2., 1., 2.]),
 array([1., 1., 0., 0., 0., 1., 0.]))

In [10]:
def x0_S(s):
    A, x, y = solve_Ay_S(s)
    return round(x[0])  # clean up mantisa garbage
#

In [11]:
x0_S("00111")

3

In [12]:
def Z(s):
    for i in range(len(s)):
        if s[i] == "0":
            yield i
#

In [13]:
list(Z("00111"))

[0, 1]

In [14]:
def a_b_c(s):
    a = len(s)
    b = 0
    for bit in s:
        if bit == "0":
            b += 1
    c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in enumerate(Z(s)))
    return (a,b,c)

In [15]:
a_b_c("00111")

(5, 2, 5)

In [16]:
def val_a_b_c(a_b_c):
    a, b, c = a_b_c
    f = Fraction( ((2**a) - c), (3**b) )
    return (f.numerator, f.denominator)
#

In [17]:
val_a_b_c((5, 2, 5))

(3, 1)

In [18]:
def val_s(s):
    return val_a_b_c(a_b_c(s))
#

In [19]:
val_s("00111")

(3, 1)

In [24]:
def s_from_abc(a: int, b: int, c: int) -> str | None:
    """
    Given (a,b,c), reconstruct the binary string s of length a
    that satisfies:
        c = sum_{j=0}^{b-1} 3^{b-1-j} * 2^{i_j}
    Returns None if no valid binary string exists.
    """
    if b == 0:
        return "1" * a  # no zeros → all ones

    frac_tup = val_a_b_c((a, b, c))
    if frac_tup[1] != 1:
        return None

    x = frac_tup[0]
    P = []
    for i in range(a):
        if x & 1 == 0:
            P.append("1")
            x = x//2
        else:
            P.append("0")
            x = (3*x + 1)//2
    return "".join(P)
#

In [25]:
s_from_abc(5, 2, 5)

'00111'

In [36]:
for i in range(300):
    x = random.randint(20, 1000)
    s = collatzPath(x)
    a, b, c = a_b_c(s)
    ss = s_from_abc(a, b, c)
    if s != ss:
        print((x, s, ss))
#

(976, '111101100011110111', None)
(498, '101001001100011011100110010110111', None)
(555, '0010101111110010110111', None)
(816, '111100110110010110111', None)
(265, '010001010110000011100100010000101100010010000001100001110101011101100011110111', None)
(655, '00001110000010010101111011000010100100010000101100010010000001100001110101011101100011110111', None)
(727, '00011100110110101011100010110111', None)
(847, '000011010111000111111111', None)
(801, '0101001110000000101110110110110010110111', None)
(561, '010111000011011100110010110111', None)
(271, '00001111100101011100010110111', None)
(970, '1011100100010000101100010010000001100001110101011101100011110111', None)
(92, '1100011110111', None)
(299, '001010110000011100100010000101100010010000001100001110101011101100011110111', None)
(751, '00001000010011000111111000011100100010000101100010010000001100001110101011101100011110111', None)
(291, '001111010000010100100010000101100010010000001100001110101011101100011110111', None)
(728, '111

In [37]:
a_b_c('111101100011110111')

(18, 5, 24976)

In [40]:
a_b_c('110110110110110111')

(18, 5, 26020)

In [20]:
def generate_s(a):
    if a == 0:
        return ["1"]
    else:
        return product('10', repeat=a)
#

In [21]:
list(generate_s(3))

[('1', '1', '1'),
 ('1', '1', '0'),
 ('1', '0', '1'),
 ('1', '0', '0'),
 ('0', '1', '1'),
 ('0', '1', '0'),
 ('0', '0', '1'),
 ('0', '0', '0')]

In [22]:
def multiplyS_2(s):
    return "1" + s
#

In [23]:
def collatzPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#


In [24]:
collatzPath(6)

'100111'

In [30]:
s_example = collatzPath(6) + "01"
val_s(s_example)

(6, 1)

In [31]:
# multiply by 2 example
s_example = "00111"
val_a_b_c(a_b_c(s_example)), val_a_b_c(a_b_c(multiplyS_2(s_example)))

((3, 1), (6, 1))

In [32]:
def shiftZ(z, t):
    return list(map(int, (np.array(z)+t)))
#

In [34]:
s_example = collatzPath(13)
z = list(Z(s_example))
zz = shiftZ(z, 2)
z, zz

([0, 3], [2, 5])

In [35]:
def binary_str(d):
    if d<0 or d>3:
        raise ValueError("d must be between 0..3")
    db = bin(d)[2:]
    if len(db) == 1:
        db = "0" + db
    return db
#

In [36]:
list(map(binary_str, [0, 1, 2, 3]))

['00', '01', '10', '11']

In [37]:
def chooseR(T, b, d):
    if d == 1:
        H = [3]
    elif d == 2:
        H = [1]
    elif d == 3:
        H = [1,3]
        
    goal = (3**b)*(2**T)*d
    sum_ = 0
    for i in range(len(H)):
        for j in H:
            sum_ = sum_ + (3**b)*(2**(T+j))
        print((sum_, goal, sum_ / goal))


In [38]:
chooseR(5, 2, 1)

(2304, 288, 8.0)


In [39]:
chooseR(6, 2, 1)

(4608, 576, 8.0)


In [40]:
def decompose_c(c: int, b: int):
    print("decompose_c(%d, %d)"%(c, b))
    """
    Attempt to decompose integer c into the form:
        c = sum_{j=0}^{b-1} 3^{b-1-j} * 2^{z_j},
    where z_0 < z_1 < ... < z_{b-1} and z_j are nonnegative integers.

    Returns:
        list of exponents [z_0, ..., z_{b-1}] if successful,
        None if no valid decomposition exists.
    """
    if c < 0 or b <= 0:
        return None

    r = c
    z_values = []

    # Work backwards from j = b-1 down to 0
    for j in reversed(range(b)):
        # Find a power of two t = 2^z satisfying t ≡ r (mod 3)
        residue = r % 3
        # Powers of two mod 3 cycle: [1, 2, 1, 2, ...]
        found = False
        for z in range(0, 64):  # arbitrary upper bound
            t = 1 << z
            if t % 3 == residue and t <= r:
                if (r - t) % 3 == 0:  # next remainder must be integer
                    z_values.append(z)
                    r = (r - t) // 3
                    found = True
                    break
        if not found:
            return None

    # After loop, we should have exactly b elements and r == 0
    if r == 0:
        # z_values were appended in reverse order (j=b-1..0)
        z_values.reverse()
        return z_values
    return None

In [41]:
# Expect [0,1]
decompose_c(5, 2)

decompose_c(5, 2)


[0, 1]

In [34]:
def decompose_c(c: int, b: int, max_z=200):
    """
    Attempt to decompose integer c into the form:
        c = sum_{j=0}^{b-1} 3^{b-1-j} * 2^{z_j},
    where 0 <= z_0 < z_1 < ... < z_{b-1} are integers.

    Returns: list [z_0,...,z_{b-1}] if successful, otherwise None.
    max_z bounds the search for powers-of-two exponents.
    """
    if c < 0 or b < 0:
        return None
    if b == 0:
        return 0 if c == 0 else None

    # Work on a copy
    r = c
    z_rev = []  # we'll collect z_{b-1},...,z_0

    for j in range(b-1, -1, -1):
        # At this step we must pick t = 2^{z} such that t ≡ r (mod 3)
        # and 0 <= t <= r. Try powers of two in increasing order.
        residue = r % 3
        found = False
        # loop over candidate exponents (0..max_z)
        for z in range(0, max_z+1):
            t = 1 << z
            if t > r:
                break
            if t % 3 != residue:
                continue
            # test divisibility after removing t
            if (r - t) % 3 == 0:
                # valid choice
                z_rev.append(z)
                r = (r - t) // 3
                found = True
                break
        if not found:
            return None

    # finished, r must be zero
    if r != 0:
        return None

    # z_rev holds [z_{b-1},...,z_0]; reverse to get ascending list
    z_list = list(reversed(z_rev))
    # ensure strictly increasing
    if any(z_list[i] >= z_list[i+1] for i in range(len(z_list)-1)):
        # if not strictly increasing, decomposition invalid
        # (but in many examples it will be strictly increasing)
        # You could allow nonstrict if needed, but generator expects increasing indices.
        return None
    return z_list
#
def apply_4n_plus_d_search(a: int, b: int, c: int, d: int,
                          b_max: int = 30, max_z: int = 200
                          ) -> Optional[Tuple[int,int,int,List[int]]]:
    """
    Attempt to perform the 4n + d operation on the lattice-point represented by (a,b,c).
    Search for b' in range(b, b_max+1). For each b' compute
        c' = 2^{a+2} - (4*n + d) * 3^{b'}
    and test if c' >= 0 and decompose_c(c', b') succeeds.

    Returns (a_new, b_new, c_new, zeros_list) or None if not found up to b_max.
    """
    if b < 0 or a < 0:
        return None
    # ensure (a,b,c) is consistent (optional)
    if (2**a - c) % (3**b) != 0:
        raise ValueError("Input (a,b,c) not consistent: (2^a - c) not divisible by 3^b")

    n = (2**a - c) // (3**b)
    a_new = a + 2
    target = 4 * n + d

    for b_new in range(1, b_max + 1):
        c_new = 2**a_new - target * (3**b_new)
        if c_new < 0:
            # if c_new negative for this b_new, increasing b_new will only make RHS larger (3^b_new grows),
            # so c_new will become even more negative; however for some cases a larger b_new might still be needed
            # because the decomposition requires more digits. We therefore continue (but one can break
            # early in certain monotonic cases).
            continue
        zeros = decompose_c(c_new, b_new, max_z=max_z)
        if zeros is not None:
            # success: return the new triple and the zero indices
            return a_new, b_new, c_new, zeros

    return None
#

In [35]:
def apply_4n_plus_d_to_s(s, d):
    a, b, c = a_b_c(s)
    return apply_4n_plus_d_search(a, b, c, d, a)
    

In [36]:
apply_4n_plus_d_to_s("00111", 1)

In [37]:
collatzPath(3), collatzPath(13), collatzPath(40)

('00111', '0110111', '1110111')

In [38]:
a_b_c(collatzPath(3)), a_b_c(collatzPath(13)), a_b_c(collatzPath(40) +"01")

((5, 2, 5), (7, 2, 11), (9, 2, 152))

In [ ]:
for i in range(100):
    u = a_b_c(collatzPath(i))
    v = a_b_c(collatzPath(4*i + 1))
    if u[1]  == v[1]:
        continue
    if v[0] <= u[0]:
        w = u[0] - v[0]
        if (w % 2) == 0:
            w = w//2
            suffix = "".join(["01"]*w)
            vv = a_b_c(collatzPath(4*i + 1) + suffix)
            if u[1] == v[1]:
                continue
    print(i)
#

In [ ]:
def cc_sum(zz):
    N = len(zz)
    total = 0
    for i in range(N):
        total = total + (3**(N-i-1))*((2**zz[i]))
    return total
#

def op4nPlusd(s, d):
    a, b, c = a_b_c(s)
    k = val_a_b_c((a, b, c))

    aa = a + 2
    kk = 4*k
    cc = (4*c) - (3**b)*d
    
    z = Z(s)
    T = a + 3

    if d == 1:
        zz = [T]
    elif d == 2:
        zz = [T+1]
    else:
        zz = [T, T+1]
    zz.extend(shiftZ(z, T+2))

    ccc = cc_sum(zz)
    ccc = ccc//(2**T)
    print(ccc)
    # ccc = 4cc + (3^bb)d
    for bb in range(0, aa+2, 1):
        x = ccc - (3**bb)*d
        if x < 0:
            continue
        if x % 4 == 0:
            print("%d - %d is divisible by 4"%(ccc, (3**bb)*d))
            cc = x//4
            z_values = decompose_c(cc, bb)
            if z_values is None:
                continue
            else:
                # We have a solution
                V = ["1"]*aa
                for idx in z_values:
                    V[idx] = "0"
                print("".join(V))
                break

    if bb == (aa+1):
        raise ValueError("Failed to find a x divisible by 4 to give a bb")

    print((aa, bb, cc))
    val = val_a_b_c((aa, bb, cc))
    print(val)
    

In [ ]:
op4nPlusd("00111", 1)

In [ ]:
val_s("0011111")

In [ ]:
for aa in range(5, 10, 1):
    print(val_a_b_c((aa, 2, 5)))

In [26]:
for i in range(4, 100, 1):
    print(i)
    u = a_b_c(collatzPath(i))
    v = a_b_c(collatzPath(4*i + 1))
    if u[1] == v[1]:
        continue
    print((i, u, v))

4
(4, (2, 0, 0), (9, 3, 53))
5
6
(6, (6, 2, 10), (16, 7, 10861))
7
8
(8, (3, 0, 0), (18, 8, 45631))
9
10
(10, (5, 1, 2), (69, 40, 91831526537371570871))
11
12
(12, (7, 2, 20), (17, 7, 23909))
13
14
(14, (12, 5, 694), (22, 10, 828511))
15
16
(16, (4, 0, 0), (19, 8, 97823))
17
18
(18, (14, 6, 3262), (73, 42, 1457146759138888205135))
19
20
(20, (6, 1, 4), (16, 6, 6487))
21
22
(22, (11, 4, 266), (21, 9, 345365))
23
24
(24, (8, 2, 40), (75, 43, 5938006025687065179749))
25
26
(26, (8, 2, 22), (26, 12, 11307559))
27
28
(28, (13, 5, 1388), (10, 2, 7))
29
30
(30, (13, 5, 902), (61, 34, 287904023554039103))
31
32
(32, (5, 0, 0), (77, 44, 24080281070142797796623))
33
34
(34, (10, 3, 106), (58, 32, 34366610279009527))
35
36
(36, (15, 6, 6524), (74, 42, 3023712507409288769479))
37
38
(38, (15, 6, 5066), (25, 11, 6450941))
39
40
(40, (7, 1, 8), (63, 35, 1168293275915822981))
41
42
(42, (7, 1, 2), (33, 16, 1315038743))
43
44
(44, (12, 4, 532), (22, 9, 710413))
45
46
(46, (12, 4, 370), (30, 14, 188892

In [27]:
collatzPath(24), collatzPath(4*24 + 1)

('11100111',
 '010100011000010100100010000101100010010000001100001110101011101100011110111')

In [29]:
collatzPath(24) + "".join(["01"]*33)

'11100111010101010101010101010101010101010101010101010101010101010101010101'

In [30]:
a_b_c(collatzPath(24) + "".join(["01"]*33))

(74, 35, 18888265174396204861816)